In [53]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import glob as glob
import math

# This script is to have create new dataframes with 44 matching harmonized bilateral region suvrs for both adni and a4

* Note in the path for calling the data, replace the /home/mlauber with home/yourfolder

In [54]:
#load and analyze amyloid positive a4 data 
merged_a4_at_amypos = pd.read_csv("/home/mlauber/jad2024/data_paths_and_cleaning/data/intermediate_data/a4/merged_a4_at_amy_pos.csv")

In [55]:
print(merged_a4_at_amypos)

           BID         update_stamp_x       ligand  centiloid  \
0    B10423472  2018-12-06 00:00:00.0  Florbetapir    80.8687   
1    B10693248  2018-12-06 00:00:00.0  Florbetapir   113.8210   
2    B10709603  2018-12-06 00:00:00.0  Florbetapir    40.5933   
3    B10904689  2018-12-06 00:00:00.0  Florbetapir   126.6360   
4    B11089276  2018-12-06 00:00:00.0  Florbetapir    35.1012   
..         ...                    ...          ...        ...   
368  B92372599  2019-04-19 00:00:00.0  Florbetapir    40.5933   
369  B97841288  2019-04-19 00:00:00.0  Florbetapir    71.7152   
370  B98032541  2019-04-19 00:00:00.0  Florbetapir    55.2389   
371  B99033399  2019-04-19 00:00:00.0  Florbetapir    53.4082   
372  B99860801  2019-04-19 00:00:00.0  Florbetapir    35.1012   

     Mean_3rd_Ventricle  Mean_4th_Ventricle  Mean_Brain_Stem  \
0              0.845577            0.864285         0.911925   
1              1.110908            0.963089         1.000662   
2              1.041388    

#### notes from a4 doumentation (stanford pipeline)
- created using freesurfer recon- all 
- seems like a4 ones all start with Mean_

In [56]:
#check how many cols have bi_ prefix 
bi_reg_count = sum(col.startswith('bi_') for col in merged_a4_at_amypos.columns)
print(bi_reg_count)
#50 bilateral regions 

50


In [57]:
filtered_columns = [col for col in merged_a4_at_amypos.columns if col.startswith('bi_')]
column_list = list(filtered_columns)
a4_col_list = pd.DataFrame({'Column Names': column_list})
# a4_col_list.to_csv("../a4/a4_bi_columns.csv", index=False)

In [58]:
#load and analyze adni data
merged_adni_at_amypos = pd.read_csv("/home/mlauber/jad2024/data_paths_and_cleaning/data/intermediate_data/adni/merged_adni_at_amy_pos.csv")

In [59]:
print(merged_adni_at_amypos['HIPPOCAMPUS_SUVR'])

0      1.596
1      1.562
2      1.169
3      1.145
4      1.328
       ...  
393    1.296
394    1.338
395    0.971
396    1.311
397    1.042
Name: HIPPOCAMPUS_SUVR, Length: 398, dtype: float64


In [60]:
#volume columns are named like ******_VOLUME so drop these 
drop_cols = [col for col in merged_adni_at_amypos.columns if col.endswith('_VOLUME')]
merged_adni_at_amypos = merged_adni_at_amypos.drop(columns=drop_cols)

In [61]:
#also drop choroid 
#drop adni startswith("CHOROID")
adni_drop_cols = [col for col in merged_adni_at_amypos.columns if col.startswith('CHOROID')]
merged_adni_at_amypos = merged_adni_at_amypos.drop(columns=adni_drop_cols)

In [62]:
#this is without the volume 
print(merged_adni_at_amypos.columns)

Index(['RID', 'TRACER_x', 'CENTILOIDS', 'LONIUID', 'SITEID', 'PTID',
       'SCANDATE', 'PROCESSDATE', 'TRACER_y', 'META_TEMPORAL_SUVR',
       ...
       'RIGHT_CEREBELLUM_CORTEX_SUVR', 'RIGHT_CEREBELLUM_WHITE_MATTER_SUVR',
       'RIGHT_CEREBRAL_WHITE_MATTER_SUVR', 'RIGHT_CHOROID_PLEXUS_SUVR',
       'RIGHT_HIPPOCAMPUS_SUVR', 'RIGHT_PALLIDUM_SUVR', 'RIGHT_PUTAMEN_SUVR',
       'RIGHT_THALAMUS_PROPER_SUVR', 'RIGHT_VENTRALDC_SUVR', 'update_stamp'],
      dtype='object', length=158)


In [63]:
suffixes_to_drop = ['_Ventricle', '_plexus','_vessel','_Chiasm','_hypointensities','WM','CSF','Unknown','_y','_x',"Matter","Vent"]
for suffix in suffixes_to_drop:
    a4_drop_cols = [col for col in merged_a4_at_amypos.columns if col.endswith(suffix)]
    merged_a4_at_amypos = merged_a4_at_amypos.drop(columns=a4_drop_cols)

In [64]:
#seems like the volume columns are named like Volume_**** so drop these columns 
a4_drop_cols = [col for col in merged_a4_at_amypos.columns if col.startswith('Volume_')]
merged_a4_at_amypos = merged_a4_at_amypos.drop(columns=a4_drop_cols)

In [65]:
merged_a4_at_amypos.head(5)

,BID,ligand,centiloid,Mean_Brain_Stem,Mean_CC_Anterior,Mean_CC_Central,Mean_CC_Mid_Anterior,Mean_CC_Mid_Posterior,Mean_CC_Posterior,Mean_Left_Accumbens_area,...,bi_precentral,bi_precuneus,bi_rostralanteriorcingulate,bi_rostralmiddlefrontal,bi_superiorfrontal,bi_superiorparietal,bi_superiortemporal,bi_supramarginal,bi_temporalpole,bi_transversetemporal
0,B10423472,Florbetapir,80.8687,0.911925,0.857168,0.733042,0.748900,0.663917,0.866686,1.199217,...,0.955041,1.079061,1.170967,1.084401,1.046561,0.904519,1.056903,0.986939,1.098317,0.949163
1,B10693248,Florbetapir,113.8210,1.000662,0.908223,0.862122,0.873952,0.837348,0.964523,1.289822,...,0.958097,1.114343,1.062407,1.174429,0.979405,0.958415,1.093653,1.146316,1.118879,1.065898
2,B10709603,Florbetapir,40.5933,1.007361,0.853996,0.872356,0.826985,0.782240,0.966260,1.433114,...,0.920471,1.039841,1.029535,0.987615,0.897984,0.900524,1.055535,1.035770,1.069306,0.981632
3,B10904689,Florbetapir,126.6360,0.920387,0.971428,0.823842,0.924871,0.792009,0.918345,1.227138,...,0.912223,1.217753,1.065679,0.993498,1.011460,1.141170,1.077074,1.141837,1.087700,0.963342
4,B11089276,Florbetapir,35.1012,1.035534,0.875824,0.751278,0.736648,0.804294,0.880089,1.459700,...,1.049212,1.160123,1.158528,1.085493,1.048737,1.015478,1.092743,1.098332,1.040324,1.066399


#### now drop the left and right duplicates 
- also with left right from a4 stanford doumentation "We additionally created bilateral regions taking a volume weighted average across hemispheres. These are provided for in the bi_* columns" so maybe just use these
- in adni data it's going to be "RIGHT_**" OR "LEFT_*" OR "CTX_RH" OR "CTX_LH" 
- for adni some regions like orbiotfrontal gyrus, there are 3: orbiotfrontal gyrus, LEFT_ORBITOFRONTAL_GYRUS, and RIGHT_ etc so just take the one without left and right 


In [67]:
#find the adni brain regions list without "RIGHT_**" OR "LEFT_*" OR "CTX_RH" OR "CTX_LH" 
prefixes_to_remove = ['RIGHT_', 'LEFT_', 'CTX_RH', 'CTX_LH']
for prefix in prefixes_to_remove:
    adni_drop_cols = [col for col in merged_adni_at_amypos.columns if col.startswith(prefix)]
    merged_adni_at_amypos = merged_adni_at_amypos.drop(columns=adni_drop_cols)

In [68]:
merged_adni_at_amypos_bi = merged_adni_at_amypos
# merged_adni_at_amypos_bi.to_csv("/home/mlauber/jad2024/data_paths_and_cleaning/data/intermediate_data/adni/merged_adni_at_amy_pos_bi.csv")
print(merged_adni_at_amypos_bi)

      RID TRACER_x  CENTILOIDS   LONIUID  SITEID        PTID    SCANDATE  \
0      31      FBP        97.0  I1596172      23  023-S-0031  2019-04-23   
1     112      FBP        89.0  I1600753     127  127-S-0112  2018-08-13   
2     120      FBP        32.0  I1596585      27  027-S-0120  2017-09-13   
3     210      FBP        54.0  I1599039      68  068-S-0210  2018-03-13   
4     377      FBP        85.0  I1598072      37  037-S-0377  2019-06-05   
..    ...      ...         ...       ...     ...         ...         ...   
393  6580      FBB       142.0  I1603427     941  941-S-6580  2020-10-09   
394  6581      FBB        28.0  I1603410     941  941-S-6581  2018-09-21   
395  6586      FBB        40.0  I1638643     135  135-S-6586  2022-10-26   
396  7100      FBB        50.0  I1634248      33  033-S-7100  2022-10-19   
397  7105      FBP       112.0  I1640013      35  035-S-7105  2022-10-25   

    PROCESSDATE TRACER_y  META_TEMPORAL_SUVR  ...  AMYGDALA_SUVR  \
0    2022-09-02    

In [69]:
#drop a4 columns that have the string "lh" "rh" or "Right" or "Left"
pattern = r'(_lh_|_rh_|Left|Right)'
lr_cols = merged_a4_at_amypos.filter(regex=pattern, axis=1)
merged_a4_at_amypos_bi = merged_a4_at_amypos.drop(columns=lr_cols)
print(merged_a4_at_amypos_bi)

           BID       ligand  centiloid  Mean_Brain_Stem  Mean_CC_Anterior  \
0    B10423472  Florbetapir    80.8687         0.911925          0.857168   
1    B10693248  Florbetapir   113.8210         1.000662          0.908223   
2    B10709603  Florbetapir    40.5933         1.007361          0.853996   
3    B10904689  Florbetapir   126.6360         0.920387          0.971428   
4    B11089276  Florbetapir    35.1012         1.035534          0.875824   
..         ...          ...        ...              ...               ...   
368  B92372599  Florbetapir    40.5933         0.919593          0.696205   
369  B97841288  Florbetapir    71.7152         0.837669          0.714328   
370  B98032541  Florbetapir    55.2389         1.046703          0.694822   
371  B99033399  Florbetapir    53.4082         0.940477          0.921984   
372  B99860801  Florbetapir    35.1012         0.924592          0.844704   

     Mean_CC_Central  Mean_CC_Mid_Anterior  Mean_CC_Mid_Posterior  \
0     

#### make all the a4 col names uppercase 

In [71]:
merged_a4_at_amypos_bi.columns = [col.upper() for col in merged_a4_at_amypos_bi.columns]
print(merged_a4_at_amypos_bi.columns)

Index(['BID', 'LIGAND', 'CENTILOID', 'MEAN_BRAIN_STEM', 'MEAN_CC_ANTERIOR',
       'MEAN_CC_CENTRAL', 'MEAN_CC_MID_ANTERIOR', 'MEAN_CC_MID_POSTERIOR',
       'MEAN_CC_POSTERIOR', 'BI_ACCUMBENS_AREA', 'BI_AMYGDALA', 'BI_CAUDATE',
       'BI_CEREBELLUM_CORTEX', 'BI_HIPPOCAMPUS', 'BI_PALLIDUM', 'BI_PUTAMEN',
       'BI_THALAMUS', 'BI_VENTRALDC', 'BI_BANKSSTS',
       'BI_CAUDALANTERIORCINGULATE', 'BI_CAUDALMIDDLEFRONTAL', 'BI_CUNEUS',
       'BI_ENTORHINAL', 'BI_FRONTALPOLE', 'BI_FUSIFORM', 'BI_INFERIORPARIETAL',
       'BI_INFERIORTEMPORAL', 'BI_INSULA', 'BI_ISTHMUSCINGULATE',
       'BI_LATERALOCCIPITAL', 'BI_LATERALORBITOFRONTAL', 'BI_LINGUAL',
       'BI_MEDIALORBITOFRONTAL', 'BI_MIDDLETEMPORAL', 'BI_PARACENTRAL',
       'BI_PARAHIPPOCAMPAL', 'BI_PARSOPERCULARIS', 'BI_PARSORBITALIS',
       'BI_PARSTRIANGULARIS', 'BI_PERICALCARINE', 'BI_POSTCENTRAL',
       'BI_POSTERIORCINGULATE', 'BI_PRECENTRAL', 'BI_PRECUNEUS',
       'BI_ROSTRALANTERIORCINGULATE', 'BI_ROSTRALMIDDLEFRONTAL',
      

In [72]:
#drop the mean columns from a4
a4_mean_drop_cols = [col for col in merged_a4_at_amypos_bi.columns if col.startswith('MEAN')]
merged_a4_at_amypos_bi = merged_a4_at_amypos_bi.drop(columns=a4_mean_drop_cols)

In [73]:
#remove the BI prefix from A4 
merged_a4_at_amypos_bi.columns = merged_a4_at_amypos_bi.columns.str.replace("BI_",'')

In [74]:
print(merged_a4_at_amypos_bi.columns)

Index(['BID', 'LIGAND', 'CENTILOID', 'ACCUMBENS_AREA', 'AMYGDALA', 'CAUDATE',
       'CEREBELLUM_CORTEX', 'HIPPOCAMPUS', 'PALLIDUM', 'PUTAMEN', 'THALAMUS',
       'VENTRALDC', 'BANKSSTS', 'CAUDALANTERIORCINGULATE',
       'CAUDALMIDDLEFRONTAL', 'CUNEUS', 'ENTORHINAL', 'FRONTALPOLE',
       'FUSIFORM', 'INFERIORPARIETAL', 'INFERIORTEMPORAL', 'INSULA',
       'ISTHMUSCINGULATE', 'LATERALOCCIPITAL', 'LATERALORBITOFRONTAL',
       'LINGUAL', 'MEDIALORBITOFRONTAL', 'MIDDLETEMPORAL', 'PARACENTRAL',
       'PARAHIPPOCAMPAL', 'PARSOPERCULARIS', 'PARSORBITALIS',
       'PARSTRIANGULARIS', 'PERICALCARINE', 'POSTCENTRAL',
       'POSTERIORCINGULATE', 'PRECENTRAL', 'PRECUNEUS',
       'ROSTRALANTERIORCINGULATE', 'ROSTRALMIDDLEFRONTAL', 'SUPERIORFRONTAL',
       'SUPERIORPARIETAL', 'SUPERIORTEMPORAL', 'SUPRAMARGINAL', 'TEMPORALPOLE',
       'TRANSVERSETEMPORAL'],
      dtype='object')


In [75]:
#drop CTX prefix and drop SUVR suffix
merged_adni_at_amypos_bi.columns = merged_adni_at_amypos_bi.columns.str.replace("CTX_",'').str.replace('_SUVR','')

In [76]:
print(merged_adni_at_amypos_bi.columns) #in adni it's thalamus_proper so need to drop the proper for it to match 

Index(['RID', 'TRACER_x', 'CENTILOIDS', 'LONIUID', 'SITEID', 'PTID',
       'SCANDATE', 'PROCESSDATE', 'TRACER_y', 'META_TEMPORAL', 'ENTORHINAL',
       'INFERIORCEREBELLUM', 'CEREBRAL_WHITE_MATTER', 'BRAINSTEM',
       'CC_ANTERIOR', 'CC_CENTRAL', 'CC_MID_ANTERIOR', 'CC_MID_POSTERIOR',
       'CC_POSTERIOR', 'CSF', 'VENTRICLE_3RD', 'VENTRICLE_4TH', 'BANKSSTS',
       'CAUDALANTERIORCINGULATE', 'CAUDALMIDDLEFRONTAL', 'CUNEUS',
       'FRONTALPOLE', 'FUSIFORM', 'INFERIORPARIETAL', 'INFERIORTEMPORAL',
       'INSULA', 'ISTHMUSCINGULATE', 'LATERALOCCIPITAL',
       'LATERALORBITOFRONTAL', 'LINGUAL', 'MEDIALORBITOFRONTAL',
       'MIDDLETEMPORAL', 'PARACENTRAL', 'PARAHIPPOCAMPAL', 'PARSOPERCULARIS',
       'PARSORBITALIS', 'PARSTRIANGULARIS', 'PERICALCARINE', 'POSTCENTRAL',
       'POSTERIORCINGULATE', 'PRECENTRAL', 'PRECUNEUS',
       'ROSTRALANTERIORCINGULATE', 'ROSTRALMIDDLEFRONTAL', 'SUPERIORFRONTAL',
       'SUPERIORPARIETAL', 'SUPERIORTEMPORAL', 'SUPRAMARGINAL', 'TEMPORALPOLE',
     

In [77]:
merged_adni_at_amypos_bi= merged_adni_at_amypos_bi.rename(columns={'THALAMUS_PROPER': 'THALAMUS'})

In [78]:
print(merged_a4_at_amypos_bi.columns)

Index(['BID', 'LIGAND', 'CENTILOID', 'ACCUMBENS_AREA', 'AMYGDALA', 'CAUDATE',
       'CEREBELLUM_CORTEX', 'HIPPOCAMPUS', 'PALLIDUM', 'PUTAMEN', 'THALAMUS',
       'VENTRALDC', 'BANKSSTS', 'CAUDALANTERIORCINGULATE',
       'CAUDALMIDDLEFRONTAL', 'CUNEUS', 'ENTORHINAL', 'FRONTALPOLE',
       'FUSIFORM', 'INFERIORPARIETAL', 'INFERIORTEMPORAL', 'INSULA',
       'ISTHMUSCINGULATE', 'LATERALOCCIPITAL', 'LATERALORBITOFRONTAL',
       'LINGUAL', 'MEDIALORBITOFRONTAL', 'MIDDLETEMPORAL', 'PARACENTRAL',
       'PARAHIPPOCAMPAL', 'PARSOPERCULARIS', 'PARSORBITALIS',
       'PARSTRIANGULARIS', 'PERICALCARINE', 'POSTCENTRAL',
       'POSTERIORCINGULATE', 'PRECENTRAL', 'PRECUNEUS',
       'ROSTRALANTERIORCINGULATE', 'ROSTRALMIDDLEFRONTAL', 'SUPERIORFRONTAL',
       'SUPERIORPARIETAL', 'SUPERIORTEMPORAL', 'SUPRAMARGINAL', 'TEMPORALPOLE',
       'TRANSVERSETEMPORAL'],
      dtype='object')


#### now find over lapping regions (with same name)

In [79]:
#rename BID to rid in a4 to make it consistent with adni 
merged_a4_at_amypos_bi = merged_a4_at_amypos_bi.rename(columns={'BID':'RID'})
#rename ligand col to tracer 
merged_a4_at_amypos_bi = merged_a4_at_amypos_bi.rename(columns={'LIGAND':'TRACER'})
merged_a4_at_amypos_bi = merged_a4_at_amypos_bi.rename(columns={'CENTILOID':'CENTILOIDS'})


In [80]:
#get a list of regions in both dataframes (44 common regions)
common_regions = list(set(merged_adni_at_amypos_bi.columns)& set(merged_a4_at_amypos_bi.columns))
print(len(common_regions))
#now filter dfs based on common regions 
merged_a4_at_amypos_bi_harm = merged_a4_at_amypos_bi[common_regions]
merged_adni_at_amypos_bi_harm = merged_adni_at_amypos_bi[common_regions]
merged_a4_at_amypos_bi_harm = merged_a4_at_amypos_bi_harm.set_index('RID')
merged_adni_at_amypos_bi_harm = merged_adni_at_amypos_bi_harm.set_index('RID')


45


In [81]:
#round to 3 decimal points so its like the adni 
merged_a4_at_amypos_bi_harm = merged_a4_at_amypos_bi_harm.round(3)
print(merged_a4_at_amypos_bi_harm)

           LATERALOCCIPITAL  PARSOPERCULARIS  LATERALORBITOFRONTAL  PUTAMEN  \
RID                                                                           
B10423472             0.896            1.106                 1.232    1.341   
B10693248             1.105            1.140                 1.187    1.419   
B10709603             1.054            1.064                 1.167    1.583   
B10904689             1.195            1.081                 1.176    1.214   
B11089276             1.066            1.175                 1.270    1.679   
...                     ...              ...                   ...      ...   
B92372599             1.237            0.997                 1.125    1.453   
B97841288             1.200            1.135                 1.345    1.651   
B98032541             1.041            1.223                 1.229    1.308   
B99033399             1.016            1.023                 1.123    1.315   
B99860801             0.938            0.873        

In [82]:
print(merged_adni_at_amypos_bi_harm)

      LATERALOCCIPITAL  PARSOPERCULARIS  LATERALORBITOFRONTAL  PUTAMEN  \
RID                                                                      
31               1.270            1.158                 1.157    1.461   
112              1.194            1.687                 1.583    1.951   
120              1.257            1.088                 1.062    1.524   
210              1.681            1.381                 1.222    1.482   
377              1.520            1.456                 1.557    2.034   
...                ...              ...                   ...      ...   
6580             1.384            1.244                 1.316    1.669   
6581             1.220            1.145                 1.056    1.648   
6586             1.179            1.108                 1.105    1.876   
7100             1.111            1.015                 1.062    1.211   
7105             2.742            1.163                 1.175    1.805   

      PARSORBITALIS  ISTHMUSCINGULATE

In [83]:
#save the new csvs that have the same regions across both 
merged_a4_at_amypos_bi_harm.to_csv('/home/mlauber/jad2024/data_paths_and_cleaning/data/intermediate_data/a4/merged_a4_at_amy_pos_bi_harm.csv')
merged_adni_at_amypos_bi_harm.to_csv('/home/mlauber/jad2024/data_paths_and_cleaning/data/intermediate_data/adni/merged_adni_at_amy_pos_bi_harm.csv')

check they both have the same number of regions 

In [84]:
print(merged_a4_at_amypos_bi_harm.shape)
print(merged_adni_at_amypos_bi_harm.shape)

(373, 44)
(398, 44)
